# Exploratory Data Analysis for Chunking

Retrieval Augementation and Generation (RAG) systems are increasingly common. Generally it is easy to implement a proof of concept that passes the infamous "vibe" check fairly quickly, whilst this might get an engineer 70-80% of the way, as with any data driven system, the devil really is in the details when it comes to squeezing out the extra performance that gives end users confidence.

The questions we'll look at include:
- What length should my chunks be?
- Should all of my chunks be the same size?
- What's the distribution of my chunk lengths?
- How should I consider the relationship between my chunk length, and the context window in my generation step?
- Do my chunks make sense in the context of my business problem?
- Is semantic purity important for my chunks? How do I measure it?

This notebook aims to give direction to data professionals in how they might approach evaluating, and selecting a chunking methodology for a RAG system. 

This notebook does NOT intend on covering end to end evaluation of RAG systems, as that is a much broader topic that dives into information retrieval. Resources for tuning those elements of a RAG system can be found [here]().

Let's load some data and get started!


## Loading a corpus
First things first, let's load some text data to work with. Let's go with the [pubmed summarisation dataset](https://huggingface.co/datasets/ccdv/pubmed-summarization). We'll download from hugging face, but for simplicity we'll convert the dataset to pandas, which most data pro's are familiar with.


In [ ]:
from datasets import load_dataset
from uuid import uuid4
from pprint import pprint
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import tiktoken as tk
import random
import json
from multiprocessing import Pool

# Set to pubmed or arxiv
publication = 'pubmed'

dataset = load_dataset(f'ccdv/{publication}-summarization',split='validation',trust_remote_code=True)

# Convert to a pandas dataframe and do some housekeeping
ds = dataset.to_pandas()
ds['doc_id'] = [str(uuid4()) for _ in range(len(ds))]
ds['article_len'] = ds['article'].apply(lambda x: len(x.split()))
ds['abstract_len'] = ds['abstract'].apply(lambda x: len(x.split()))

ds.head()

Let's take a look at the distribution of article and abstract lengths.

In [ ]:
ds.describe()

In [ ]:
# Lets plot a histogram with seaborn

# Create a figure and a 1x2 grid of subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first histogram on the first subplot
sns.histplot(ds['article_len'], bins=50, kde=True, ax=axs[0])
axs[0].set_title('Article Length')

# Plot the second histogram on the second subplot
sns.histplot(ds['abstract_len'], bins=50, kde=True, ax=axs[1])
axs[1].set_title('Abstract Length')

# Display the plots
plt.tight_layout()
plt.show()

Before we even start diving deeper, we can see the heavy right skew of the article length distribution. For now, let's take a closer look at the length percentiles to see what might make dor a good cutoff.

In [ ]:
ds.describe(percentiles=[0.75,0.8, 0.9,0.95, 0.99])

The problem is definitely in the high end of town, with a jump from 10k to 112k in number of words for the last percentile. Let's take a closer look at the raw data and see if we can tell what's going on.

In [ ]:
# Get the indices of the articles with the longest length
longest_articles = ds['article_len'].nlargest(5).index
for idx in longest_articles:
    print(f'Article Length: {ds["article_len"][idx]}\n')
    print(ds['article'][idx]+'\n')
    print('\n')

At first glance, it appears that the two main causes of long documents are either:
- LaTeX package inclusions (mathematical formatting for scientific documents)
- Data tables (from pharmaceutical research by the look of it)

In practice we would want to spend more time understanding the drivers behind the outliers, and address as many as possible. Given this is an **information retrieval** promlem we want to avoid excluding valid records. 

However, for the purposes of this exercise we will focus on the LaTeX issue. Data tables could be solved through an application of difference pdf cracking techniques (e.g. [Azure Document Intelligence](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept-retrieval-augmented-generation?view=doc-intel-4.0.0)), but that is out of scope for this notebook.

Let's remove the the lines which include LaTeX and take another look at the adjusted distributions.

In [ ]:
from topic_utils.general import remove_latex_packages

# Remove LaTeX package inclusions from the articles
ds['article'] = ds['article'].apply(remove_latex_packages)

# Recalculate article lengths
ds['article_len'] = ds['article'].apply(lambda x: len(x.split()))

display(ds.describe(percentiles=[0.75,0.8, 0.9,0.95, 0.99]))

This has made a difference, but 97k is still very large. For now, let's exclude the longer documents, storing them in another dataframe for analysis later.

Once we remove the odd docs, we'll check our distribution again to make sure that we now have something workable.

In [ ]:
from topic_utils.general import remove_over_percentile

#apply helper function from utils module
ds_99pct, ds_outliers = remove_over_percentile(ds, 'article_len', .99)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first histogram on the first subplot
sns.histplot(ds_99pct['article_len'], bins=50, kde=True, ax=axs[0])
axs[0].set_title('Article Length')

# Plot the second histogram on the second subplot
sns.histplot(ds_99pct['abstract_len'], bins=50, kde=True, ax=axs[1])
axs[1].set_title('Abstract Length')

# Display the plots
plt.tight_layout()
plt.show()

display(ds_99pct.describe())

## Let's get chunking!

Now we know the distribution of our document lengths, we can start to look at how to approach chunking. Remember our questions were:

- What length should my chunks be?
- Should all of my chunks be the same size?
- What's the distribution of my chunk lengths?
- How should I consider the relationship between my chunk length, and the context window in my generation step?
- Do my chunks make sense in the context of my business problem?
- Is semantic purity important for my chunks? How do I measure it?

Let's start with the first question. There won't be a one size fits all answer here, but there will be "non functional" considerations. We now know that our mean article length is about `3000` words, and the max is ~`10,000` words. What does that tell us?
1. the number of calls that we need to make to an embedding service will be `3000 / chunk_size'
2. Whilst we could fit entire documents into the generation step of RAG using models with a large context window (e.g. Claude 3 - Opus) - we probably don't want to

> **Note: Words and tokens**: *Despite feeling like LLMs converse in our language, there's a few things that go in behind the scenes that translate our verbiage into something an algorithm understands. Firstly, the text is `tokenized`, which means words are split into a list of `tokens`. Think of this a bit like stemming in NLP. For shorter words, the ratio of tokens to words can be 1:1 (i.e. the word = the token), but for longer, or more complex words the ratio can be far higher. These lists are then converted into numerical vectors that the algorithm can understand. For a given corpus, we could work out the exact ratio - in fact, let's do that!*



In [ ]:
# run the tokeniser over the articles and abstracts and store the results in the DataFrame
encoding = tk.encoding_for_model('gpt-3.5-turbo')

article_tokens = ds_99pct['article'].apply(encoding.encode)
abstract_tokens = ds_99pct['abstract'].apply(encoding.encode)


# check if columns already exist
if 'article_tokens' in ds_99pct.columns:
    ds_99pct = ds_99pct.drop(columns=['article_tokens'])
if 'abstract_tokens' in ds_99pct.columns:
    ds_99pct = ds_99pct.drop(columns=['abstract_tokens'])

ds_99pct = ds_99pct.assign(article_tokens=article_tokens, abstract_tokens=abstract_tokens)
ds_99pct['article_tk_len'] = ds_99pct['article_tokens'].apply(lambda x: len(x))
ds_99pct['abstract_tk_len'] = ds_99pct['abstract_tokens'].apply(lambda x: len(x))


Let's look at the distributions again. You could use a histogram, but I prefer box plots, or violin pots if I'm feeling fancy. These are great as the show the univariate stats (mean/median etc.) but also provide the same "distribution curve" visual that you'd get from a histogram. 

In [ ]:
# Calculate the mean, median and mode for the token lengths
article_token_mean = ds_99pct['article_tk_len'].mean()
article_token_median = ds_99pct['article_tk_len'].median()
article_token_mode = ds_99pct['article_tk_len'].mode()[0]
abstract_tokens_mean = ds_99pct['abstract_tk_len'].mean()
abstract_tokens_median = ds_99pct['abstract_tk_len'].median()
abstract_tokens_mode = ds_99pct['abstract_tk_len'].mode()[0]

print("***----***---***---***")
print("This is what we're working with:\n")
print(f'Mean Article Tokens: {article_token_mean}')
print(f'Median Article Tokens: {article_token_median}')
print(f'Mode Article Tokens: {article_token_mode}')
print(f'Mean Abstract Tokens: {abstract_tokens_mean}')
print(f'Median Abstract Tokens: {abstract_tokens_median}')
print(f'Mode Abstract Tokens: {abstract_tokens_mode}')
print("***----***---***---***")

# Let's plot the token lengths as violin plots as two panels and call out the mean, median and mode
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first violin plot on the first subplot
sns.violinplot(y=ds_99pct['article_tk_len'], ax=axs[0])
axs[0].set_title('Article Tokens')
axs[0].axhline(article_token_mean, color='red', linestyle='--', label='Mean')
axs[0].axhline(ds_99pct['article_tk_len'].median(), color='green', linestyle='--', label='Median')
axs[0].axhline(ds_99pct['article_tk_len'].mode()[0], color='blue', linestyle='--', label='Mode')
axs[0].legend()

# Plot the second violin plot on the second subplot
sns.violinplot(y=ds_99pct['abstract_tk_len'], ax=axs[1])
axs[1].set_title('Abstract Tokens')
axs[1].axhline(abstract_tokens_mean, color='red', linestyle='--', label='Mean')
axs[1].axhline(ds_99pct['abstract_tk_len'].median(), color='green', linestyle='--', label='Median')
axs[1].axhline(ds_99pct['abstract_tk_len'].mode()[0], color='blue', linestyle='--', label='Mode')
axs[1].legend()

# Display the plots
plt.tight_layout()
plt.show()


Let's go with our mean article token length of 3800, and article length of 3000 - which gives us a ratio of aproximately 1.25 or 5:4 for our specific corpus. 

# Why did do we care about this?

How many records do we want to include in our Augmentation step when constructing the generation prompt? Say we're using GPT-35-Turbo, we have aprx 4000 tokens to play with (for an explanation of tokens see [this](https://www.tokencounter.io/) excellent resource). This is both input and output.
Let's assume we have a prompt template which is a total of 500 tokens, including our guardrails, instructions and any other boiler plate commentary that needs to be input to the generation step. Say we then allow for up to 500 tokens in a response. This leaves us with 3000 tokens (or 2400 words) to play with. If we assume a chunk size (in number of words) of 400, that gives us ~6 records in our retrieval step. In fact, this might be a good starting point. Why not try baseline chunking with with this as a starting point.

In [ ]:
# Let's now apply this to our dataset
from topic_utils.general import chunk_string_with_overlap

# Create a new DataFrame with each chunk as a separate row
chunks = []
doc_ids = []
chunk_ids = []
for idx, row in ds_99pct.iterrows():
    article_chunks = chunk_string_with_overlap(input_text=row['article'], chunk_length=400, overlap=50)
    chunks.extend(article_chunks)
    doc_ids.extend([row['doc_id']] * len(article_chunks))
    chunk_ids.extend([f"{row['doc_id']}-{i+1}" for i in range(len(article_chunks))])

ds_chunked = pd.DataFrame({'doc_id': doc_ids, 'chunk_id': chunk_ids, 'chunks': chunks})

# Worl out the average number of chunks per document
avg_chunks_per_doc = ds_chunked.groupby('doc_id').size().mean()
print(f'Average number of chunks per document: {avg_chunks_per_doc}')

## Now we have some chunks - let's start having fun

We have some operational concerns to deal with next. Before we can work out if the chunking strategy is any good, we will need to embed the chunks, and store them in a vector database. We'll then need to come up with some basic questions and answers to test the system - GPT4 is ideal for this. 

> Note: Whilst we could include a variety of search configurations to test, here we're only concerned with the relevance of the chunks compared to the question. We'll simplify the problem by simply measuring the cosine similarity of the question and answer for now.

For each document, we'll use GPT4 to generate 5 questions and answers as our test set. To save on time and money, we'll reduce the number of articles we're dealing with down to 50.

In [ ]:
# select a random 50 unique doc_ids and subset both the ds_99pct and new_df dataframes using these IDs
random.seed(42)
random_doc_ids = random.sample(list(ds_99pct['doc_id'].unique()), 50)

# Subset the DataFrames
ds_subset = ds_99pct[ds_99pct['doc_id'].isin(random_doc_ids)]

ds_chunked_subset = ds_chunked[ds_chunked['doc_id'].isin(random_doc_ids)]

# Submit the articles to GPT-3.5-turbo for Q&A creation 
def generate_qa_prompt(article):
    prompt = f"""
    Given the following article, generate 5 Question/Answer pairs that could be used to test a student's understanding of the material:

    Article:\n
    {article}\n

    The output should be a list of dictionaries, with each question/answer pair structured as follows:
    {{
        "question": "What is the capital of France?",
        "answer": "Paris"
    }}

    Only provide the data in as describes, do not include any other information in the output.
    Ensure that the output is formatted as a list of dictionaries.
    Do not include markdown or any other formatting in the output e.g. no ```json.
    Do not generate questions that are too similar to each other.
    Do not generate questions that require external knowledge.
    """
    return prompt

prompts = [generate_qa_prompt(article) for article in ds_subset['article']]


In [ ]:
# Multithreaded ~ 13x faster (3mins vs 40mins)
#TODO: Add a check for the file, if it exists read it in, else do the hit the endpoint

from topic_utils.openai_utils import general_prompt, create_client
from topic_utils.general import convert_to_dict

client = create_client()
multi_threading = True

# CHeck if file already exists
if os.path.exists('data/qa_pairs.jsonl'):
    print("File exists, reading in...")

    with open('data/qa_pairs.jsonl', 'r') as f:
        qa_pairs = json.load(f)
    


else:
    # Note changing this to 3.5 without implementing guardrails may result in malformed results...
    model = 'gpt-4'

    if multi_threading == True:
        def process_article(article):
            return general_prompt(client, generate_qa_prompt(article), model=model)

        with Pool() as pool:
            results_multiprocessing = pool.map(process_article, ds_subset['article'])

    else:
        results = [general_prompt(client, prompt, model=model) for prompt in prompts]

    # Save the results to a file
    qa_pairs = convert_to_dict(results)

    with open('data/qa_pairs.jsonl', 'w') as f:
        json.dump(qa_pairs, f, indent=4)


## Setting up the Retrieval step
### Check in
Where are we up to:
- We have a good understanding of our corpus and have done some housekeeping
- We understand tokens, and how our chunks are related to context length
- We have a chunked data set to act as a baseline
- We have generated some ground truth data to evaluate our chunks

What do we need to do next:
- Embed our data
- Store it in a vector database
- Query the db using our ground truth questions
- Generate a final response
- Run it through an evaluation framework
- ITERATE

Let's start with embeddings. There are many different embedding models out there. I'm going to assume that given we've used Azure Open AI, that we can also access an embedding model through the same resource. Be sure to have things configured in your `.env` file.


In [ ]:
import ast
# Takes about 2 mins to run
def generate_embeddings(text, model="text-embedding-ada-002"):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# if a column exists delete it
if 'ada_v2' in ds_chunked_subset.columns:
    ds_chunked_subset = ds_chunked_subset.drop(columns=['ada_v2'])

# Check if file exists
if os.path.exists('data/chunked_embeddings.csv'):
    print("File exists, reading in...")

    ds_chunked_subset = pd.read_csv('data/chunked_embeddings.csv')
    ds_chunked_subset['ada_v2'] = ds_chunked_subset['ada_v2'].apply(ast.literal_eval)
    
else:

    if multi_threading == True:
        with Pool() as pool:
            results_multiprocessing = pool.map(generate_embeddings, ds_chunked_subset['chunks'])
            ds_chunked_subset['ada_v2'] = results_multiprocessing

    else:
        ds_chunked_subset['ada_v2'] = ds_chunked_subset["chunks"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

    # Save the results to a file
    ds_chunked_subset.to_csv('data/chunked_embeddings.csv', index=False)

In [ ]:
type(ds_chunked_subset['ada_v2'][0])

Now lets store these in a vector database. To keep things simple, we've elected to use ChromaDB, which is the "SQLLite" of the vector database world. Note, we could have skipped the above step and used ChromaDB's OpenAIEmbedding interface. We'll need to set this up to embed queries regardless.

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_type="azure",
                api_version=os.getenv("OPENAI_API_VERSION"),
                model_name="text-embedding-ada-002"
            )

In [ ]:
from chromadb import Client
chroma_client = Client()

# if chroma_client.list_collections()[0].name == "baseline_pubmed_articles":
#     chroma_client.delete_collection(name="baseline_pubmed_articles")

collection = chroma_client.create_collection(name="baseline_pubmed_articles",embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})

collection.add(
    embeddings=ds_chunked_subset['ada_v2'].tolist(),
    documents=ds_chunked_subset['chunks'].tolist(),
    metadatas=[{"doc_id": doc_id} for doc_id in ds_chunked_subset['doc_id']],
    ids=ds_chunked_subset['chunk_id'].tolist()
)

In [ ]:
results = collection.query(
    query_texts=["What is the main diagnostic criterion related to Levodopa (LD) responsiveness in Parkinson's Disease (PD)?"],
    n_results=5
)

results

We can see that the results object contains:
- The chunk IDs
- The distances (in our case, cosine similarity) between the question and the chunks
- The metadata we injected with the prompt - in our case the doc_ids (in another notebook we can talk about *hybrid search*)
- The embeddings are set to none by default as typically they're not particularly useful here
- The actual chunk contents

Different vector DBs will return different responses, but most will be similar to what we have here.

If we were looking to tune the search params, we might pause at this step to see if there's a way to increase the relevance of the results to the query; but we'll look at that another time! For now, let's move on to using these responses to actually answer the user's question.

## Augementation Step
Now we need to create a query which takes those responses, and injects them into a final generation prompt to be submitted to an LLM. Let's try it out on a single Q&A pair first to make sure everything is as expected.

In [ ]:
context = '\n'.join(results['documents'][0])
question = 'What is the main diagnostic criterion related to Levodopa (LD) responsiveness in Parkinson\'s Disease (PD)?'

generation_prompt = f"""
You provide answers to questions based on information available. You give precise answers to the question asked.
You do not answer more than what is needed. You are always exact to the point. You Answer the question using the provided context.
If the answer is not contained within the given context, say 'I dont know.'. 
The below context is an excerpt from a report or data.
Answer the user question using only the data provided in the sources below.

CONTEXT:
{context}
 

QUESTION:
{question}

ANSWER:
"""

model = 'gpt-4'

final_result = general_prompt(client, generation_prompt, model=model)

pprint(final_result)

In [ ]:
def ask_rag(question, client, model, collection):
    results = collection.query(
        query_texts=[question],
        n_results=5
    )

    context = '\n'.join(results['documents'][0])
    generation_prompt = f"""
    You provide answers to questions based on information available. You give precise answers to the question asked.
    You do not answer more than what is needed. You are always exact to the point. You Answer the question using the provided context.
    If the answer is not contained within the given context, say 'I dont know.'. 
    The below context is an excerpt from a report or data.
    Answer the user question using only the data provided in the sources below.

    CONTEXT:
    {context}
    

    QUESTION:
    {question}

    ANSWER:
    """

    return general_prompt(client, generation_prompt, model=model)

In [ ]:
ask_rag("What is the main diagnostic criterion related to Levodopa (LD) responsiveness in Parkinson's Disease (PD)?", client, model, collection)

In [ ]:
# Extract questions and answers from qa_pairs
questions =[pair['question'] for pair in qa_pairs]
answers = [pair['answer'] for pair in qa_pairs]

# Create a DataFrame with the questions and answers
qa_df = pd.DataFrame({'question': questions, 'answer': answers})

In [ ]:
# If things are taking too long (e.g. more than 10 minutes), you can switch to the smaller model
model = 'gpt-4' #'gpt-35-turbo-16k'

if os.path.exists(f'data/qa_df-{model}.csv'):
    print("File exists, reading in...")

    qa_df = pd.read_csv(f'data/qa_df-{model}.csv')

else:
    def ask_rag_wrapper(question):
        return ask_rag(question, client, model, collection)

    if multi_threading == True:
        with Pool() as pool:
            results_multiprocessing = pool.map(ask_rag_wrapper, qa_df['question'])
        qa_df['mt_answer'] = results_multiprocessing

    else:
        qa_df['response'] = qa_df['question'].apply(lambda x: ask_rag(x, client, model, collection))

#write out to CSV
qa_df.to_csv(f'data/qa_df-{model}.csv', index=False)

# Evaluation

In [ ]:
from azure.identity import AzureCliCredential
from azure.core.credentials import AzureKeyCredential

from azure.ai.resources.client import AIClient
from azure.ai.generative.evaluate import evaluate
from mlflow import MlflowClient


# project details
SUBSCRIPTION_ID = "bc6dcbb4-93d4-4ee9-8e1c-2a265ede8e06"
RESOURCE_GROUP = "promptflow"
PROJECT_NAME = "agromova-8777"
AZUREAIPROJECT_KEY = "44027ae2fb564731b9ab5978ce1bdfd0"
AZUREAIPROJECT_ENDPOINT = "https://agazureaistudi0026803630.openai.azure.com/"


# Create Azure AI Studio client
ai_client = AIClient(
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    project_name=PROJECT_NAME,
    credential=AzureCliCredential(),
)
# Get the default Azure Open AI connection for your project
ai_client.get_default_aoai_connection().set_current_environment()

In [ ]:
result = evaluate( 
    evaluation_name="my-qa-eval-with-data", #name your evaluation to view in AI Studio
    data=qa_df, # data to be evaluated
    task_type="qa",
    metrics_list=["gpt_groundedness","gpt_relevance","gpt_coherence","gpt_fluency","gpt_similarity","hate_unfairness","sexual","violence","self_harm"],
    model_config= { #for AI-assisted metrics, need to hook up AOAI GPT model for doing the measurement
            "api_version": "2023-05-15",
            "api_base": AZUREAIPROJECT_ENDPOINT,
            "api_type": "azure",
            "api_key": AZUREAIPROJECT_KEY,
            "deployment_id": "gpt-4"
    },
    data_mapping={
        "question":"question", #column of data providing input to model
        "context":"context", #column of data providing context for each input
        "answer":"answer", #column of data providing output from model
        "ground_truth":"groundtruth" #column of data providing ground truth answer, optional for default metrics
        },
    output_path="./myevalresults", #optional: save evaluation results .jsonl to local folder path 
    tracking_uri=client.tracking_uri #optional: if configured with AI client, evaluation gets logged to AI Studio
)